In [ ]:
!python -m pip install --upgrade pip

In [57]:
# image name preferably should start with sagemaker
!docker build -t sagemaker_feature_encoder8 -f Dockerfile .

Sending build context to Docker daemon  79.36kB
Step 1/6 : FROM python:3.7
 ---> 5cda39795abb
Step 2/6 : COPY requirements.txt ./
 ---> Using cache
 ---> cb0502180872
Step 3/6 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> b8cd9ebdcc1a
Step 4/6 : COPY serve /usr/local/bin
 ---> 489a781d9ae6
Step 5/6 : RUN chmod +x /usr/local/bin/serve
 ---> Running in d440280ad473
Removing intermediate container d440280ad473
 ---> a8a246da487c
Step 6/6 : EXPOSE 8080
 ---> Running in 9999896bab4e
Removing intermediate container 9999896bab4e
 ---> 6a4ce9422455
Successfully built 6a4ce9422455
Successfully tagged sagemaker_feature_encoder8:latest


In [58]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_feature_encoder8
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  sagemaker_feature_encoder8
Account:  119174016168
Region: us-east-1
ECR Repository:  119174016168.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_feature_encoder8:latest
Login Succeeded
The push refers to repository [119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_feature_encoder8]
b8d30430824a: Preparing
2c8089a3bade: Preparing
9697b36946da: Preparing
b9e5d8fe6251: Preparing
eef135e35b6e: Preparing
a0db21004f62: Preparing
0b53caaeb40b: Preparing
1cad4dc57058: Preparing
4ff8844d474a: Preparing
b77487480ddb: Preparing
cd247c0fb37b: Preparing
cfdd5c3bd77e: Preparing
870a241bfebd: Preparing
a0db21004f62: Waiting
0b53caaeb40b: Waiting
1cad4dc57058: Waiting
4ff8844d474a: Waiting
b77487480ddb: Waiting
cd247c0fb37b: Waiting
cfdd5c3bd77e: Waiting
870a241bfebd: Waiting
2c8089a3bade: Pushed
b8d30430824a: Pushed
b9e5d8fe6251: Pushed
a0db21004f62: Pushed
eef135e35b6e: Pushed
1cad4dc57058: Pushed
0b53caaeb40b: Pushed
cd2

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [ ]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

### Imports 

In [59]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3

In [60]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_feature_encoder8'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
image_uri

'119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_feature_encoder8:latest'

In [61]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = f'feature-encoder-{current_timestamp}'

In [62]:
!tar -czf encoder.tar.gz -C ./data .

In [63]:
default_bucket = session.default_bucket()
default_bucket

'sagemaker-us-east-1-119174016168'

In [64]:
!aws s3 cp encoder.tar.gz s3://{default_bucket}/

upload: ./encoder.tar.gz to s3://sagemaker-us-east-1-119174016168/encoder.tar.gz


In [65]:
model_artifacts_location = f's3://{default_bucket}/encoder.tar.gz'

In [66]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             "Image": image_uri, 
                                             "Mode": "SingleModel", 
                                             "ModelDataUrl": model_artifacts_location}], 
                                         ExecutionRoleArn=role)

print(f'Model Arn: {response["ModelArn"]}')

Model Arn: arn:aws:sagemaker:us-east-1:119174016168:model/feature-encoder-2022-12-15-17-55-26


In [67]:
ep_config_name = "sm-feature-encoder" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    
    
    ProductionVariants=[
        {
            "VariantName": "v1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.xlarge",
            'InitialInstanceCount': 2,
                        'InitialVariantWeight': 1,}
    ]
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:119174016168:endpoint-config/sm-feature-encoder2022-12-15-17-55-27


In [68]:
response = sagemaker_client.create_endpoint(
              EndpointName       = ep_config_name + '1',
              EndpointConfigName = ep_config_name)
response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint/sm-feature-encoder2022-12-15-17-55-271',
 'ResponseMetadata': {'RequestId': 'e5042bd3-cfec-42fb-be1b-0b321e6fb49d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5042bd3-cfec-42fb-be1b-0b321e6fb49d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Thu, 15 Dec 2022 17:55:27 GMT'},
  'RetryAttempts': 0}}

In [69]:


sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')



In [72]:
%%time

endpoint_name = ep_config_name + '1'
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b"10-19-2021,145,24,22,14,usa,65+",
    ContentType="text/csv",
)
response

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 112 ms


{'ResponseMetadata': {'RequestId': 'ea8a0a60-c949-48f4-aecc-da91344d9037',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea8a0a60-c949-48f4-aecc-da91344d9037',
   'x-amzn-invoked-production-variant': 'v1',
   'date': 'Thu, 15 Dec 2022 17:57:39 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '128'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'v1',
 'Body': <botocore.response.StreamingBody at 0x7fbfd5019610>}

In [73]:


predicted_label = response['Body'].read().decode('utf-8').strip()
predicted_label



'0.04209896249002394,0.41690962099125367,0.3142857142857143,0.25925925925925924,0.208955223880597,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0'